In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('C:\\Users\\anjel\\Downloads\\SCMA')

In [3]:
ipl_bbb = pd.read_csv('IPL_ball_by_ball_updated till 2024.csv',low_memory=False)

In [33]:
ipl_bbb.head(2)

,Match id,Date,Season,Batting team,Bowling team,Innings No,Ball No,Bowler,Striker,Non Striker,runs_scored,extras,type of extras,score,score/wicket,wicket_confirmation,wicket_type,fielders_involved,Player Out
0,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.1,P Kumar,SC Ganguly,BB McCullum,0,1,legbyes,1,1/0,0,NaN,NaN,NaN
1,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.2,P Kumar,BB McCullum,SC Ganguly,0,0,NaN,1,1/0,0,NaN,NaN,NaN


In [4]:
ipl_salary = pd.read_excel('IPL SALARIES 2024.xlsx')

In [5]:
ipl_salary.head(2)

,Player,Salary,Rs,international,iconic
0,Abhishek Porel,20 lakh,20,0,NaN
1,Anrich Nortje,6.5 crore,650,1,NaN


In [6]:
grouped_data = ipl_bbb.groupby(['Season', 'Innings No', 'Striker','Bowler']).agg({'runs_scored': sum, 'wicket_confirmation':sum}).reset_index()

In [7]:
player_runs = grouped_data.groupby(['Season', 'Striker'])['runs_scored'].sum().reset_index()
player_wickets = grouped_data.groupby(['Season', 'Bowler'])['wicket_confirmation'].sum().reset_index()

In [29]:
player_runs[player_runs['Season']=='2023'].sort_values(by='runs_scored',ascending=False)

,Season,Striker,runs_scored
2423,2023,Shubman Gill,890
2313,2023,F du Plessis,730
2311,2023,DP Conway,672
2433,2023,V Kohli,639
2443,2023,YBK Jaiswal,625
...,...,...,...
2404,2023,RP Meredith,0
2372,2023,Mohsin Khan,0
2307,2023,DG Nalkande,0
2429,2023,TU Deshpande,0


In [8]:
top_run_getters = player_runs.groupby('Season').apply(lambda x: x.nlargest(3, 'runs_scored')).reset_index(drop=True)
bottom_wicket_takers = player_wickets.groupby('Season').apply(lambda x: x.nlargest(3, 'wicket_confirmation')).reset_index(drop=True)
print("Top Three Run Getters:")
print(top_run_getters)
print("Top Three Wicket Takers:")
print(bottom_wicket_takers)

Top Three Run Getters:
     Season          Striker  runs_scored
0   2007/08         SE Marsh          616
1   2007/08        G Gambhir          534
2   2007/08    ST Jayasuriya          514
3      2009        ML Hayden          572
4      2009     AC Gilchrist          495
5      2009   AB de Villiers          465
6   2009/10     SR Tendulkar          618
7   2009/10        JH Kallis          572
8   2009/10         SK Raina          528
9      2011         CH Gayle          608
10     2011          V Kohli          557
11     2011     SR Tendulkar          553
12     2012         CH Gayle          733
13     2012        G Gambhir          590
14     2012         S Dhawan          569
15     2013       MEK Hussey          733
16     2013         CH Gayle          720
17     2013          V Kohli          639
18     2014       RV Uthappa          660
19     2014         DR Smith          566
20     2014       GJ Maxwell          552
21     2015        DA Warner          562
22     2015

In [9]:
ipl_year_id = pd.DataFrame(columns=["id", "year"])
ipl_year_id["id"] = ipl_bbb["Match id"]
ipl_year_id["year"] = pd.to_datetime(ipl_bbb["Date"], dayfirst=True).dt.year

In [10]:
#create a copy of ipl_bbbc dataframe
ipl_bbbc= ipl_bbb.copy()

In [11]:
ipl_bbbc['year'] = pd.to_datetime(ipl_bbb["Date"], dayfirst=True).dt.year

In [12]:
ipl_bbbc[["Match id", "year", "runs_scored","wicket_confirmation","Bowler",'Striker']].head()

,Match id,year,runs_scored,wicket_confirmation,Bowler,Striker
0,335982,2008,0,0,P Kumar,SC Ganguly
1,335982,2008,0,0,P Kumar,BB McCullum
2,335982,2008,0,0,P Kumar,BB McCullum
3,335982,2008,0,0,P Kumar,BB McCullum
4,335982,2008,0,0,P Kumar,BB McCullum


In [13]:
import scipy.stats as st

def get_best_distribution(data):
    dist_names = ['alpha','beta','betaprime','burr12','crystalball',
                  'dgamma','dweibull','erlang','exponnorm','f','fatiguelife',
                  'gamma','gengamma','gumbel_l','johnsonsb','kappa4',
                  'lognorm','nct','norm','norminvgauss','powernorm','rice',
                  'recipinvgauss','t','trapz','truncnorm']
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        # Applying the Kolmogorov-Smirnov test
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))
    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value
    print("\nBest fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))
    return best_dist, best_p, params[best_dist]

In [14]:
total_run_each_year = ipl_bbbc.groupby(["year", "Striker"])["runs_scored"].sum().reset_index()

In [48]:
total_run_each_year.sort_values(["year", "runs_scored"], ascending=False, inplace=True)
print(total_run_each_year)

      year          Striker  runs_scored
2549  2024       RD Gaikwad          509
2589  2024          V Kohli          500
2470  2024  B Sai Sudharsan          418
2502  2024         KL Rahul          406
2555  2024          RR Pant          398
...    ...              ...          ...
58    2008         L Balaji            0
66    2008   M Muralitharan            0
75    2008         MM Patel            0
107   2008      S Sreesanth            0
136   2008           U Kaul            0

[2598 rows x 3 columns]


In [15]:
list_top_batsman_last_three_year = {}
for i in total_run_each_year["year"].unique()[:3]:
    list_top_batsman_last_three_year[i] = total_run_each_year[total_run_each_year.year == i][:3]["Striker"].unique().tolist()

In [16]:
list_top_batsman_last_three_year

{2008: ['A Chopra', 'A Kumble', 'A Mishra'],
 2009: ['A Chopra', 'A Flintoff', 'A Kumble'],
 2010: ['A Kumble', 'A Mishra', 'A Mithun']}

In [56]:
import warnings
warnings.filterwarnings('ignore')
runs = ipl_bbbc.groupby(['Striker','Match id'])[['runs_scored']].sum().reset_index()

for key in list_top_batsman_last_three_year:
    for Striker in list_top_batsman_last_three_year[key]:
        print("************************")
        print("year:", key, " Batsman:", Striker)
        get_best_distribution(runs[runs["Striker"] == Striker]["runs_scored"])
        print("\n\n")

************************
year: 2024  Batsman: RD Gaikwad
p value for alpha = 2.599259711013304e-20
p value for beta = 0.02041902689492492
p value for betaprime = 0.0195037635986679
p value for burr12 = 0.46882020698395865
p value for crystalball = 0.24953646987270617
p value for dgamma = 0.1570743843120962
p value for dweibull = 0.20046582403736823
p value for erlang = 1.893799588395604e-06
p value for exponnorm = 0.4644304230917985
p value for f = 1.3560920695663998e-07
p value for fatiguelife = 1.304427037367869e-14
p value for gamma = 0.005830868576003456
p value for gengamma = 0.015331622187827243
p value for gumbel_l = 0.05546236480086464
p value for johnsonsb = 4.646964117947127e-13
p value for kappa4 = 0.006363220770325362
p value for lognorm = 1.1719355665219537e-16
p value for nct = 0.5881570496217812
p value for norm = 0.2495365180930973
p value for norminvgauss = 0.5538573365184996
p value for powernorm = 0.1788753268739085
p value for rice = 0.18287532184336575
p value for 

In [17]:
total_wicket_each_year = ipl_bbbc.groupby(["year", "Bowler"])["wicket_confirmation"].sum().reset_index()

In [18]:
total_wicket_each_year.sort_values(["year", "wicket_confirmation"], ascending=False, inplace=True)
print(total_wicket_each_year)

      year             Bowler  wicket_confirmation
1836  2024           HV Patel                   19
1875  2024       Mukesh Kumar                   15
1822  2024     Arshdeep Singh                   14
1842  2024          JJ Bumrah                   14
1876  2024  Mustafizur Rahman                   14
...    ...                ...                  ...
16    2008           CL White                    0
41    2008             K Goel                    0
43    2008          LPC Silva                    0
60    2008       Pankaj Singh                    0
90    2008        VS Yeligati                    0

[1929 rows x 3 columns]


In [19]:
list_top_bowler_last_three_year = {}
for i in total_wicket_each_year["year"].unique()[:3]:
    list_top_bowler_last_three_year[i] = total_wicket_each_year[total_wicket_each_year.year == i][:3]["Bowler"].unique().tolist()
list_top_bowler_last_three_year

{2024: ['HV Patel', 'Mukesh Kumar', 'Arshdeep Singh'],
 2023: ['MM Sharma', 'Mohammed Shami', 'Rashid Khan'],
 2022: ['YS Chahal', 'PWH de Silva', 'K Rabada']}

In [66]:
import warnings
warnings.filterwarnings('ignore')
wickets = ipl_bbbc.groupby(['Bowler','Match id'])[['wicket_confirmation']].sum().reset_index()

for key in list_top_bowler_last_three_year:
    for bowler in list_top_bowler_last_three_year[key]:
        print("************************")
        print("year:", key, " Bowler:", bowler)
        get_best_distribution(wickets[wickets["Bowler"] == bowler]["wicket_confirmation"])
        print("\n\n")

************************
year: 2024  Bowler: HV Patel
p value for alpha = 0.0002993252328930706
p value for beta = 2.777571908776589e-19
p value for betaprime = 1.7052883875145053e-30
p value for burr12 = 5.427998338605459e-15
p value for crystalball = 1.1109118198587684e-05
p value for dgamma = 4.375428528574276e-05
p value for dweibull = 1.8553295107771936e-05
p value for erlang = 5.473635282991912e-24
p value for exponnorm = 0.0002813279943461815
p value for f = 1.9012983291282487e-09
p value for fatiguelife = 1.9734428958773156e-05
p value for gamma = 1.470787431589663e-16
p value for gengamma = 1.4345058849022962e-16
p value for gumbel_l = 4.541523588271283e-05
p value for johnsonsb = 2.827201329331457e-51
p value for kappa4 = 9.177530010006471e-23
p value for lognorm = 5.2162358572043325e-22
p value for nct = 0.0001960277304576293
p value for norm = 1.1109124960635979e-05
p value for norminvgauss = 3.811196478020768e-05
p value for powernorm = 3.2186417463058256e-05
p value for r

**Relationship between the performance of a player and the salary he gets**

In [20]:
R2024 =total_run_each_year[total_run_each_year['year']==2024]

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd

ipl_bbb = pd.read_csv('IPL_ball_by_ball_updated till 2024.csv',low_memory=False)


FileNotFoundError: [Errno 2] No such file or directory: 'IPL_ball_by_ball_updated till 2024.csv'

In [21]:
#pip install fuzzywuzzy

In [22]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = ipl_salary.copy()
df_runs = R2024.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Striker'].tolist()))

# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Striker')

In [103]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          111 non-null    object 
 1   Salary          111 non-null    object 
 2   Rs              111 non-null    int64  
 3   international   111 non-null    int64  
 4   iconic          0 non-null      float64
 5   Player_Match    111 non-null    object 
 6   Matched_Player  111 non-null    object 
 7   year            111 non-null    int32  
 8   Striker         111 non-null    object 
 9   runs_scored     111 non-null    int64  
dtypes: float64(1), int32(1), int64(3), object(5)
memory usage: 8.4+ KB


In [23]:
# Calculate the correlation
correlation = df_merged['Rs'].corr(df_merged['runs_scored'])

print("Correlation between Salary and Runs:", correlation)

Correlation between Salary and Runs: 0.3514061444454122


In [34]:
import pandas as pd



In [50]:
ipl_salary = pd.read_excel(salaries_file)

In [51]:
bbb_file = 'C:\\Users\\anjel\\Downloads\\SCMA\\IPL_ball_by_ball_updated till 2024.csv'

In [52]:
ipl_bbb = pd.read_csv(bbb_file, low_memory=False)

In [53]:
# Filter data for SO Hetmyer as striker

In [54]:
hetmyer_stats = ipl_bbb[ipl_bbb['Striker'] == 'SO Hetmyer']

In [55]:
hetmyer_performance = hetmyer_stats[['Match id', 'runs_scored', 'wicket_type']]

In [56]:
hetmyer_performance = hetmyer_performance.groupby('Match id').agg({
    'runs_scored': 'sum',          # Total runs scored
    'wicket_type': lambda x: (x == 'bowled').sum()  # Total wickets bowled
}).reset_index()


In [57]:
hetmyer_performance.columns = ['Match id', 'Runs Scored', 'Wickets Taken']


In [58]:
print(hetmyer_performance.head())

   Match id  Runs Scored  Wickets Taken
0   1175356            0              0
1   1175362            5              0
2   1175366            9              0
3   1175369            1              0
4   1178429           75              0


In [66]:
print(ipl_salary.columns)

Index(['Player', 'Salary', 'Rs', 'international', 'iconic'], dtype='object')


In [69]:
ipl_salary.head()

,Player,Salary,Rs,international,iconic
0,Abhishek Porel,20 lakh,20,0,NaN
1,Anrich Nortje,6.5 crore,650,1,NaN
2,Axar Patel,9 crore,900,0,NaN
3,David Warner,6.25 crore,625,1,NaN
4,Ishant Sharma,50 lakh,50,0,NaN


In [76]:
print(hetmyer_performance.columns)


Index(['Match id', 'Runs Scored', 'Wickets Taken'], dtype='object')


In [77]:
print(ipl_salary.columns)

Index(['Player', 'Salary', 'Rs', 'international', 'iconic'], dtype='object')


In [80]:
print("Columns of IPL Salary DataFrame:")
print(ipl_salary.columns)

print("\nColumns of IPL Ball-by-Ball DataFrame:")
print(ipl_bbb.columns)

Columns of IPL Salary DataFrame:
Index(['Player', 'Salary', 'Rs', 'international', 'iconic'], dtype='object')

Columns of IPL Ball-by-Ball DataFrame:
Index(['Match id', 'Date', 'Season', 'Batting team', 'Bowling team',
       'Innings No', 'Ball No', 'Bowler', 'Striker', 'Non Striker',
       'runs_scored', 'extras', 'type of extras', 'score', 'score/wicket',
       'wicket_confirmation', 'wicket_type', 'fielders_involved',
       'Player Out'],
      dtype='object')


In [81]:
merged_data = pd.concat([ipl_salary, ipl_bbb], axis=1)

In [82]:
print("\nMerged DataFrame:")


Merged DataFrame:


In [83]:
print(merged_data.head())

           Player      Salary     Rs  international  iconic  Match id  \
0  Abhishek Porel     20 lakh   20.0            0.0     NaN    335982   
1   Anrich Nortje   6.5 crore  650.0            1.0     NaN    335982   
2      Axar Patel     9 crore  900.0            0.0     NaN    335982   
3    David Warner  6.25 crore  625.0            1.0     NaN    335982   
4   Ishant Sharma     50 lakh   50.0            0.0     NaN    335982   

         Date   Season           Batting team                 Bowling team  \
0  18-04-2008  2007/08  Kolkata Knight Riders  Royal Challengers Bangalore   
1  18-04-2008  2007/08  Kolkata Knight Riders  Royal Challengers Bangalore   
2  18-04-2008  2007/08  Kolkata Knight Riders  Royal Challengers Bangalore   
3  18-04-2008  2007/08  Kolkata Knight Riders  Royal Challengers Bangalore   
4  18-04-2008  2007/08  Kolkata Knight Riders  Royal Challengers Bangalore   

   ...  Non Striker  runs_scored extras type of extras score  score/wicket  \
0  ...  BB McC

In [84]:
print("Original DataFrame:")

Original DataFrame:


In [86]:
print(merged_data.columns)

Index(['Player', 'Salary', 'Rs', 'international', 'iconic', 'Match id', 'Date',
       'Season', 'Batting team', 'Bowling team', 'Innings No', 'Ball No',
       'Bowler', 'Striker', 'Non Striker', 'runs_scored', 'extras',
       'type of extras', 'score', 'score/wicket', 'wicket_confirmation',
       'wicket_type', 'fielders_involved', 'Player Out'],
      dtype='object')


In [92]:
comparison_data = merged_data[['Player', 'runs_scored', 'Rs']]

In [93]:
average_stats = comparison_data.groupby('Player').agg({
    'runs_scored': 'mean',
    'Rs': 'mean'
}).reset_index()

In [94]:
print(average_stats.head())

                Player  runs_scored     Rs
0          Abdul Samad          0.0  400.0
1  Abhinav Sadarangani          0.0  260.0
2       Abhishek Porel          0.0   20.0
3      Abhishek Sharma          0.0  650.0
4           Adam Zampa          0.0  150.0


In [97]:
correlation = average_stats['runs_scored'].corr(average_stats['Rs'])

In [98]:
print(f"\nCorrelation between runs_scored and Rs: {correlation}")


Correlation between runs_scored and Rs: -0.0971388355234988


In [100]:
merged_data['Player'] = merged_data['Player'].astype(str)

In [101]:
hetmyer_data = merged_data[merged_data['Player'].str.contains('Hetmyer', case=False, na=False)]

In [102]:
# Display Hetmyer's performance and salary information

In [103]:
print("\nHetmyer's Performance and Salary Information:")


Hetmyer's Performance and Salary Information:


In [106]:
print(hetmyer_data[['Player', 'Salary', 'Rs', 'international', 'iconic', 'runs_scored', 'wicket_confirmation']])

              Player     Salary     Rs  international  iconic  runs_scored  \
126  Shimron Hetmyer  8.5 crore  850.0            1.0     NaN            0   

     wicket_confirmation  
126                    0  


In [107]:
total_runs = hetmyer_data['runs_scored'].sum()

In [109]:
hetmyer_salary = hetmyer_data['Salary'].iloc[0]

In [110]:
print("\nSummary Comparison:")
print(f"Total Runs Scored by Hetmyer: {total_runs}")
print(f"Total Wickets Taken by Hetmyer: {total_wickets}")
print(f"Hetmyer's Salary: {hetmyer_salary}")


Summary Comparison:
Total Runs Scored by Hetmyer: 0
Total Wickets Taken by Hetmyer: 2/0
Hetmyer's Salary: 8.5 crore


In [112]:
total_scores = hetmyer_data['score'].sum()

In [114]:
print(f"total_scores: {total_scores}")

total_scores: 2


In [116]:
import pandas as pd

In [117]:
# Assuming you have the following data
total_runs_scored = 0
wickets_taken = 2
hetmyer_salary = 8.5  # in crore
total_scores = 2


In [118]:
data = {
    'Total Runs Scored': [total_runs_scored],
    'Wickets Taken': [wickets_taken],
    'Salary (crore)': [hetmyer_salary],
    'Total Scores': [total_scores]
}

df = pd.DataFrame(data)

In [126]:
# Calculate correlation matrix

In [131]:
correlation = average_stats['runs_scored'].corr(average_stats['Rs'])

In [132]:
import pandas as pd

# Assuming you have the following data
total_runs_scored = 0
wickets_taken = 2
hetmyer_salary = 8.5  # in crore
total_scores = 2

# Create a DataFrame for Hetmyer's performance and salary
hetmyer_data = pd.DataFrame({
    'Player': ['Hetmyer'],
    'Runs Scored': [total_runs_scored],
    'Wickets Taken': [wickets_taken],
    'Salary (crore)': [hetmyer_salary],
    'Total Scores': [total_scores]
})

# Print summary comparison
print("Summary Comparison:")
print(f"Total Runs Scored by Hetmyer: {total_runs_scored}")
print(f"Total Wickets Taken by Hetmyer: {wickets_taken}")
print(f"Hetmyer's Salary: {hetmyer_salary} crore")
print(f"Total Scores: {total_scores}")

# Calculate correlation
correlation = hetmyer_data['Runs Scored'].corr(hetmyer_data['Salary (crore)'])

# Print correlation
print(f"Correlation between Runs Scored and Salary: {correlation}")


Summary Comparison:
Total Runs Scored by Hetmyer: 0
Total Wickets Taken by Hetmyer: 2
Hetmyer's Salary: 8.5 crore
Total Scores: 2
Correlation between Runs Scored and Salary: nan


C:\Users\anjel\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\anjel\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
